In [132]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from torch.autograd import Variable
import pandas as pd
import torch.optim as optim


In [133]:
transform_image=transforms.Compose(
    [
    torchvision.transforms.GaussianBlur(kernel_size=3, sigma=(0.1,2)),
    torchvision.transforms.RandomRotation(10),
    torchvision.transforms.Grayscale(1),
     torchvision.transforms.ToTensor(),
    ]
    )

In [134]:
train_data= torchvision.datasets.CIFAR10(root="data",
                                              train=True,
                                              download=True,
                                              transform=transform_image,
                                              target_transform=None
                                              )
test_data= torchvision.datasets.CIFAR10(root="data",
                                              train=False,
                                              download=True,
                                              transform=transform_image,
                                              target_transform=None
                                              )

Files already downloaded and verified
Files already downloaded and verified


In [135]:
# Selecting classes 0, 2, 4,6 and 8 in train data
idx0 = torch.tensor(train_data.targets) == 0
idx2 = torch.tensor(train_data.targets) == 2
idx4 = torch.tensor(train_data.targets) == 4
idx6 = torch.tensor(train_data.targets) == 6
idx8 = torch.tensor(train_data.targets) == 8

train_mask= idx0 | idx2 | idx4 | idx6 | idx8
train_indices = train_mask.nonzero().reshape(-1) #a list of indices which has true for either 0, 2, 4,6 or 8

from torch.utils.data import Subset
train_subset = Subset(train_data, train_indices)  # make a subset

from torch.utils.data import DataLoader
train_data = DataLoader(train_subset, shuffle=False, batch_size=32)


In [136]:
# Selecting classes 0, 2, 4,6 and 8 in test data
idx0 = torch.tensor(test_data.targets) == 0
idx2 = torch.tensor(test_data.targets) == 2
idx4 = torch.tensor(test_data.targets) == 4
idx6 = torch.tensor(test_data.targets) == 6
idx8 = torch.tensor(test_data.targets) == 8

test_mask= idx0 | idx2 | idx4 | idx6 | idx8
test_indices = test_mask.nonzero().reshape(-1) #a list of indices which has true for either 0, 2, 4,6 or 8

from torch.utils.data import Subset
test_subset = Subset(test_data, test_indices)  # make a subset

from torch.utils.data import DataLoader
test_data = DataLoader(test_subset, shuffle=False, batch_size=32)


In [137]:
for image, label in test_data:
  print(image.shape)
  break


torch.Size([32, 1, 32, 32])


In [138]:
len(train_data), len(test_data)

(782, 157)

In [139]:
image,label = next(iter(train_data))

In [140]:
image.shape,label

(torch.Size([32, 1, 32, 32]),
 tensor([6, 4, 2, 8, 4, 2, 2, 6, 4, 6, 6, 2, 6, 4, 0, 0, 4, 0, 2, 2, 2, 2, 0, 2,
         2, 2, 4, 8, 2, 4, 8, 6]))

In [141]:
# print(label)
# #image.resize(32,32,3)
# plt.imshow(image[0].permute(1, 2, 0))
# image.shape

In [142]:
class_names= train_data.dataset.dataset.classes  #the class_names is a list containing the names of all classes
class_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [143]:
#Converting the class names to class indexes so that we can use it later
class_to_idx= train_data.dataset.dataset.class_to_idx
class_to_idx 

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [144]:
len(train_data), len(test_data)

(782, 157)

In [145]:
#setup device agnoistic code
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device


'cpu'

In [146]:
class AE(nn.Module):
    def __init__(self,input_features,output_features):
        super().__init__()
        self.encoder_hidden_layer_1 = nn.Linear(
            in_features=input_features, out_features=128
        )
        self.encoder_hidden_layer_2= nn.Linear(
            in_features=128, out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer_1= nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_hidden_layer_2 = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output_layer = nn.Linear(
            in_features=128, out_features=output_features
        )

    def encoder(self, x):
        activation = self.encoder_hidden_layer_1(x)
        activation = torch.relu(activation)
        activation = self.encoder_hidden_layer_2(activation)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        encoded = torch.relu(code)
        return encoded

    def decoder(self,x):
        activation = self.decoder_hidden_layer_1(x)
        activation = torch.relu(activation)
        activation = self.decoder_hidden_layer_2(activation)
        activation = torch.relu(activation)
        activation = self.decoder_output_layer(activation)
        decoded = torch.relu(activation)
        return decoded

    def forward(self, x):
      reconstructed= self.decoder(self.encoder(x))
      return reconstructed

In [147]:
  #create an instance of model and send it to the target device
model_0= AE(input_features=1024,
                   output_features=1024).to(device)


In [148]:
#create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.SGD(model_0.parameters(), lr=0.1)

# mean-squared error loss
loss_fn = nn.MSELoss()

In [149]:
epochs=5
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in train_data:
        # reshaping the input matrix
        batch_features = batch_features.view(-1, 1024).to(device)
        optimizer.zero_grad()
        outputs = model_0(batch_features)
        
        # compute loss & update parameter
        # print(type(loss))
        train_loss = loss_fn(outputs, batch_features)
        train_loss.backward()
        optimizer.step()
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(train_data)
    
    # display the epoch training loss
    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

epoch : 1/5, loss = 0.162838
epoch : 2/5, loss = 0.111980
epoch : 3/5, loss = 0.110335
epoch : 4/5, loss = 0.109625
epoch : 5/5, loss = 0.109333


In [150]:
from torch import nn
#building a multiclass classification model
class Clsf_1(nn.Module):
  def __init__(self, input_features, output_features, hidden_units):
    super().__init__()
    self.linear_layer_1= nn.Sequential(
        nn.Linear(in_features= input_features, out_features=hidden_units),
        nn.Sigmoid(),
        nn.Linear(in_features=hidden_units, out_features=output_features)
    )

  def forward(self,x):
    value=model_0.encoder(x)
    value=self.linear_layer_1(value)
    return value

  #create an instance of model and send it to the target device
model_2= Clsf_1(input_features=128,
                   output_features=5,
                   hidden_units=100).to(device)



In [151]:
#create a loss function and an optimizer for multi-class classification model
loss_fn= nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(), lr=0.1)


In [152]:
#Calculate accuracy
def accuracy_fn(y_true, y_pred):
  correct = torch.eq(y_true, y_pred).sum().item()
  acc= (correct/len(y_pred))*100
  return acc

In [ ]:

torch.manual_seed(42)
torch.cuda.manual_seed(42)

#set number of epochs
epochs=10

#data to target device

#loop through data
for epoch in range(epochs):
  train_loss = 0
  for batch, (X, y) in enumerate(train_data):
    X, y =X.to(device), y.to(device)


    #training
    model_2.train()

    #1. forward pass
    y_logits = model_2(X.view(-1,1024))
    y= y//2
    #2.Calculate the loss/acc
    loss = loss_fn(y_logits, y)

    train_loss = train_loss + loss
    
    #3.Optimizer zero grad
    optimizer.zero_grad()

    #4. Loss backward
    loss.backward()

    #5. optimizer step(gradient descent)
    optimizer.step()

  
  ##Testing
  model_2.eval()

  with torch.inference_mode():
    test_loss = 0
    for X, y in test_data:
      X, y= X.to(device), y
    #1. forward pass
      test_logits = model_2(X.view(-1,1024))
      # test_pred = torch.softmax(test_logits, dim=1).argmax(dim=1)
      y=y//2
      #2. calculte the loss
      test_loss = test_loss + loss_fn(test_logits, torch.tensor(y))
    
  #print out whats hapening
  train_loss = train_loss / len(train_data)
  test_loss = test_loss / len(test_data)
  #if epoch ==0:
  print(f"Epoch: {epoch} | train loss:{train_loss:.5f} | Test loss:{test_loss:.5f}   ")




<ipython-input-153-b0748fde3908>:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_loss = test_loss + loss_fn(test_logits, torch.tensor(y))


Epoch: 0 | train loss:1.58095 | Test loss:1.55442   
